In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

import requests
from bs4 import BeautifulSoup
# from sklearn.Logit_model_reloaded import score
# data_test = pd.read_csv("test_notes.csv", sep=',')

In [2]:
# nice, psg,om, lyon, monaco, lille,rennes, mtp
team_list_av = []
x_av = []
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.transfermarkt.fr/serie-a/startseite/wettbewerb/IT1"

pageTree = requests.get(page, headers=headers)
soup = BeautifulSoup(pageTree.content, 'html.parser')
body = soup.html.find('div',{'class':'large-8 columns'}).find_all('div',{'class':'box'})[1].find('div',{'class':'responsive-table'}).div.table.tbody.find_all('tr')
ixo2 = []
for g in body:

    team_list_av.append(g.find('td',{'hauptlink no-border-links show-for-small show-for-pad'}).a.text)
    ixo = g.find_all('td',{'class':'rechts show-for-small show-for-pad nowrap'})[1].text.replace(' mio. €','').replace(',','.')
    

    if 'mrd' in ixo:
        ixo2.append(float(ixo.replace(' mrd. €',''))*1000)
    else:
        ixo2.append(ixo)
        
    x_av.append(ixo2)

In [3]:
value_model_av = pd.DataFrame({'Team':team_list_av,'Value':x_av[0]})
value_model_av.to_csv('value_model_av.csv',index=False)

In [4]:
value_model_af = pd.read_csv('value_model_av.csv',sep=',')

In [5]:
x = value_model_af['Value']
list_y = []
for i in x:
    list_y.append(0.014043285470786433*i + -3.4708104763730665)

In [6]:
value_model = pd.DataFrame({'Team':value_model_af['Team'],'Value':value_model_af['Value'],'trend':list_y})

In [7]:
value_model

,Team,Value,trend
0,Juventus Turin,640.40,5.522510
1,Inter,573.30,4.580205
2,Napoli,517.75,3.800101
3,AC Milan,466.90,3.086000
4,AS Roma,418.35,2.404198
5,Atalanta BC,415.35,2.362068
6,Lazio Rome,326.60,1.115727
7,Fiorentina,248.80,0.023159
8,Sassuolo,216.90,-0.424822
9,Torino,184.13,-0.885020


In [8]:
0.013651660777911766*2000+ -2.399976874866872

24.903344680956657

In [9]:
# Récupération du modèle de Régression Logistique 'lr'
with open('model_mhsc.pkl', 'rb') as f:
    my_unpickler = pickle.Unpickler(f)
    model_logit = my_unpickler.load()

    y_pred = model_logit.predict_proba(value_model[['trend']].values) 


In [10]:
for value in y_pred[0]:
    print(value)

0.0046487623852025095
0.9953512376147975


In [11]:
predictions = [round(value) for value in y_pred[0]]
# print(ytest)
# precision = model_logit.score(value_model[['Value','trend']],predictions)
# print(np.around(precision,decimals=2)*100,'%')

In [12]:
value_model

,Team,Value,trend
0,Juventus Turin,640.40,5.522510
1,Inter,573.30,4.580205
2,Napoli,517.75,3.800101
3,AC Milan,466.90,3.086000
4,AS Roma,418.35,2.404198
5,Atalanta BC,415.35,2.362068
6,Lazio Rome,326.60,1.115727
7,Fiorentina,248.80,0.023159
8,Sassuolo,216.90,-0.424822
9,Torino,184.13,-0.885020


In [13]:
predict = []
prt_predict = []
for i in y_pred:
    prt_predict.append(i[1])
value_model['pourcentage'] = prt_predict

count = 0
proba = 6
proba_list = []
high_count = 0
for a,k,i in zip(value_model.sort_values(by='pourcentage',ascending=False)['trend'],value_model.sort_values(by='pourcentage',ascending=False)['Team'],value_model.sort_values(by='pourcentage',ascending=False)['pourcentage']):
    if (proba == 6)&(a < 4):
        proba = proba - high_count
    
    proba_list.append(np.around(i*(proba/6),decimals=3))
    print(i,np.around(i*(proba/6),decimals=3),count)
    count = count + 1
    if a > 4:
        high_count = high_count + 1
        continue
    
    elif proba < 3:
        if count == 9:
            proba = proba - 1
        else:
            continue
    else:
        proba = proba - 1
        
    
value_model['pourcentage'] = proba_list
        
for i in list(value_model['pourcentage']):
    if i < 0.5:
        predict.append(False)
    elif i > 0.5:
        predict.append(True)
    
#print(len(predict))
#print(len(y_pred[1]))
value_model['predict'] = predict


0.9953512376147975 0.995 0
0.9861602883979946 0.986 1
0.9662825475000517 0.644 2
0.9256470082232735 0.463 3
0.8488500016264601 0.283 4
0.8424298602233208 0.281 5
0.5550789582691861 0.185 6
0.2583717073012443 0.086 7
0.1711493691945485 0.057 8
0.10766445040590158 0.018 9
0.0680801273989903 0.011 10
0.05744615903850985 0.01 11
0.04362004993100425 0.007 12
0.033708216008139544 0.006 13
0.03023535250372088 0.005 14
0.029102590637690162 0.005 15
0.01870757030129745 0.003 16
0.018188024357110134 0.003 17
0.017199379642654698 0.003 18
0.010834241018343056 0.002 19


In [14]:
print('Prédictions :', value_model['Team'][0],' : ', np.around(y_pred[0][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][1],' : ', np.around(y_pred[1][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][2],' : ', np.around(y_pred[2][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][3],' : ', np.around(y_pred[3][1],decimals=2),' %')
print('Prédictions :', value_model['Team'][4],' : ', np.around(y_pred[4][1],decimals=2),' %')

Prédictions : Juventus Turin  :  1.0  %
Prédictions : Inter  :  0.99  %
Prédictions : Napoli  :  0.97  %
Prédictions : AC Milan  :  0.93  %
Prédictions : AS Roma  :  0.85  %


In [15]:
# for i,e in zip(value_model['Team'],value_model['pourcentage']):
#     print(i,e)
# MHSC 0.29
# PSG 1.0
# OM 0.89
# OL 0.97
# ASM 0.99
# LOSC 0.91
# NICE 0.77
# SRFC 0.83 

In [16]:
value_model

,Team,Value,trend,pourcentage,predict
0,Juventus Turin,640.40,5.522510,0.995,True
1,Inter,573.30,4.580205,0.986,True
2,Napoli,517.75,3.800101,0.644,True
3,AC Milan,466.90,3.086000,0.463,False
4,AS Roma,418.35,2.404198,0.283,False
5,Atalanta BC,415.35,2.362068,0.281,False
6,Lazio Rome,326.60,1.115727,0.185,False
7,Fiorentina,248.80,0.023159,0.086,False
8,Sassuolo,216.90,-0.424822,0.057,False
9,Torino,184.13,-0.885020,0.018,False


In [17]:
value_model.to_csv('model_result_serieA.csv',index=False)